In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# 確認是否有可用的GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: {device}')


Using device: cuda


In [3]:
# 讀取CSV資料，假設包含 "text" 與 "label" 欄位
df = pd.read_csv("referance.csv")

# 假設9個分類標籤是以下（請根據實際情況調整）
categories = [
    "批評選民與動員",
    "地方或政黨政治批評",
    "人身攻擊言論",
    "罷免立場表態",
    "政策與程序討論",
    "抵制與行動呼籲",
    "創意表達與諷刺",
    "國安及賣台指控",
    "事件引用論述或加油的言論",
    "其他"
]

# 將文字標籤轉成數字ID
label2id = {cat: idx for idx, cat in enumerate(categories)}
id2label = {idx: cat for cat, idx in label2id.items()}

df['label_id'] = df['label'].apply(lambda x: label2id[x])  # 將文字標籤轉數字


In [22]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)

print("訓練筆數：", len(train_df))
print("測試筆數：", len(test_df))


訓練筆數： 283
測試筆數： 71


In [23]:
model_name = "hfl/chinese-macbert-large"  # 使用更大的預訓練模型
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 將文字資料轉成 BERT 相容的輸入格式
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="longest",     # 也可以改成 longest，若想動態padding可使用 batched 方式
        truncation=True,
        pad_to_max_length=False,
        )


In [24]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df[['text', 'label_id']])
test_dataset  = Dataset.from_pandas(test_df[['text', 'label_id']])

# 進行 Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 移除多餘欄位，並指定特徵名稱
train_dataset = train_dataset.remove_columns(["text"])
train_dataset = train_dataset.rename_column("label_id", "labels")
train_dataset.set_format("torch")

test_dataset = test_dataset.remove_columns(["text"])
test_dataset = test_dataset.rename_column("label_id", "labels")
test_dataset.set_format("torch")

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


Map:   0%|          | 0/283 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [25]:
num_labels = 10  # 分類數量
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
# 將模型移動到GPU
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [26]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",       # 每個 epoch 結束後做評估
    save_strategy="epoch",            # 每個 epoch 儲存一次模型 (可根據需求調整)
    num_train_epochs=5,               # 以資料量少為前提，先試試 5 epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,    # 視硬體調整
    per_device_eval_batch_size=4,
    logging_dir="./logs",             # 訓練過程紀錄
    logging_steps=10,
    load_best_model_at_end=True,       # 在最後載入最好的模型權重
    report_to="none"                  # 禁用報告到任何平台
)



c:\Users\andyw\Desktop\桑鑼的分類\.venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    return {"accuracy": acc["accuracy"]}


In [28]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

# 取得最佳模型存放的 checkpoint 路徑
best_checkpoint_dir = trainer.state.best_model_checkpoint
print("Best model checkpoint folder:", best_checkpoint_dir)

# 可以把最佳模型另存到某個資料夾，例如 "./best_model"
best_model_dir = "./best_model"
trainer.save_model(best_model_dir)
print(f"Best model is now saved to: {best_model_dir}")

C:\Users\andyw\AppData\Local\Temp\ipykernel_6340\1215538910.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.129800,1.867146,0.323944


KeyboardInterrupt: 

In [5]:
best_model_dir = "./best_model_"
model_name = "hfl/chinese-roberta-wwm-ext"
model = AutoModelForSequenceClassification.from_pretrained(best_model_dir)
tokenizer = AutoTokenizer.from_pretrained(best_model_dir)

In [11]:
import json

all_list = []
#for i in range(48):
for i in range(2):
    #input_json_path = f"C://Users//andyw//Desktop//桑鑼的分類//fb_comments//comments//result_{i}.json"
    input_json_path = f"C://Users//andyw//Desktop//桑鑼的分類//fb_comments//result_video{i}.json"
    
    try:
        with open(input_json_path, "r", encoding="utf-8") as f:
            file_data = json.load(f) 

        row_list = []
        for element in file_data:
            row_list.append({
                "內容": element[0],
                "時間": element[1]
            })
        all_list.append(row_list)
    except:#not found
        all_list.append([])
        pass

In [12]:
offset = 0
for i in range(offset, 2):
        
    # 3. Tokenizer 處理
    # 假設 tokenizer 和 model 已經在上方載入
    # 例如： tokenizer = AutoTokenizer.from_pretrained(model_name)
    if len(all_list[i]) == 0:
        continue
    test_texts = pd.DataFrame(all_list[i])
    test_texts['內容'] = test_texts['內容'].fillna("").astype(str)
    
    input_sentences = test_texts['內容'].tolist()  # 轉為 Python list

    encoding = tokenizer(
    input_sentences, 
    padding="longest",
    truncation=True,
    max_length=512,        # common max length
    return_tensors="pt"
    )

    # 4. 推論 (Inference)
    with torch.no_grad():
        outputs = model(**encoding)
    predictions = torch.argmax(outputs.logits, dim=1).numpy()  # 這裡會是整數

    # 5. 可將整數類別轉成「文字標籤」
    predicted_labels = [id2label[p] for p in predictions]

    # 6. 將預測結果一次性寫回 DataFrame
    test_texts['分類'] = predicted_labels

    # 7. (可選) 列印出結果檢查
    for text, label in zip(test_texts['內容'], predicted_labels):
        print(f"文本: {text}\n -> 預測分類: {label}\n")

    # 8. 儲存結果到新 Excel 檔
    output_path = f"C://Users//andyw//Desktop//桑鑼的分類//result{i}.xlsx"
    test_texts.to_excel(output_path, index=False)
    print(f"已將結果儲存至：{output_path}")

文本: 連署書可以親送嗎？
 -> 預測分類: 創意表達與諷刺

文本: 高山風 您好，目前維持建議寄件到郵政信箱，但提醒可以使用掛號的方式寄出！相對保障！
 -> 預測分類: 事件引用論述或加油的言論

文本: 罷完傅。去花蓮玩
 -> 預測分類: 創意表達與諷刺

文本: 黃國欽 歡迎唷
 -> 預測分類: 創意表達與諷刺

文本: 微光 shimmer.tw 加油
 -> 預測分類: 創意表達與諷刺

文本: 上街頭發連署書會不會更有效益？讓更多人知道更好取得？畢竟有年紀的人不一定會列印
 -> 預測分類: 國安及賣台指控

文本: 洪靜蕙 會有的
 -> 預測分類: 創意表達與諷刺

文本: 洪靜蕙 規劃中
 -> 預測分類: 創意表達與諷刺

文本: 這邊有4份 很快寄出
 -> 預測分類: 創意表達與諷刺

文本: Chen Hong Hong 謝謝
 -> 預測分類: 創意表達與諷刺

文本: Chen Hong Hong 
 -> 預測分類: 抵制與行動呼籲

文本: 連署書怎麼領取???
 -> 預測分類: 事件引用論述或加油的言論

文本: 趙玉芳 需要自行下載列印https://drive.google.com/....../1IevLDmwLJx....../view......
 -> 預測分類: 創意表達與諷刺

文本: 加油加油
 -> 預測分類: 創意表達與諷刺

文本: 花蓮加油！！！
 -> 預測分類: 創意表達與諷刺

文本: 加油
 -> 預測分類: 創意表達與諷刺

文本: 加油
 -> 預測分類: 創意表達與諷刺

文本: 每天來點布農語啊！mapasnava Bunun saikin 一起加油
 -> 預測分類: 事件引用論述或加油的言論

文本: 花蓮人 加油
 -> 預測分類: 創意表達與諷刺

文本: Happy Hsieh 花蓮人加油台灣加油
 -> 預測分類: 創意表達與諷刺

文本: 請問現在遷戶籍到花蓮來得及嗎？￼
 -> 預測分類: 事件引用論述或加油的言論

文本: 加油
 -> 預測分類: 創意表達與諷刺

文本: 加油加油
 -> 預測分類: 創意表達與諷刺

文本: 江欣燕 加油加油
 -> 預測分類: 創意表達與諷刺

文本: 加油，已分享，我很前一批6月的不知道有沒有順利收到 XD
 -> 預